In [3]:
import glob
import sys
#sys.path.append('../../../msrv-xc')
#import msrvxc

import numpy as np
import pickle

from astropy.io import fits
from astropy.table import Table

from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
with open('../../../msrv-xc/grids/interp_bosz.pkl', 'rb') as f:
    interp_bosz = pickle.load(f)
    
with open('../../../msrv-xc/grids/interp_bosz_norm.pkl', 'rb') as f:
    interp_bosz_norm = pickle.load(f)
    
wvl = np.load('../../../msrv-xc/grids/wavl.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../../../msrv-xc/grids/interp_bosz.pkl'

In [4]:
def continuum_normalize(wl, fl, ivar = None, avg_size = 150, ret_cont = False):
    
    fl_norm = np.zeros(np.size(fl))
    fl_cont = np.zeros(np.size(fl))
    
    ivar_yes = 0
    if ivar is not None:
        ivar_yes = 1
        ivar_norm = np.zeros(np.size(fl))
        
    for i in range(np.size(wl)):
        wl_clip = ((wl[i]-avg_size/2)<wl) * (wl<(wl[i]+avg_size/2))
        fl_cont[i] = np.median(fl[wl_clip])
        if ivar_yes:
            ivar_norm[i] = ivar[i]*np.median(fl[wl_clip])**2
    
    fl_norm = fl/fl_cont
    
    if ret_cont:
        if ivar_yes:
            return wl, fl_norm, ivar_norm, fl_cont
        else:
            return wl, fl_norm, fl_cont
    else:
        if ivar_yes:
            return wl, fl_norm, ivar_norm
        else:
            return wl, fl_norm

In [5]:
files = glob.glob('./*.fits')

In [6]:
from tqdm import tqdm

names = []
ras = []
decs = []
rvs = []
e_rvs = []

for file in tqdm(files):
    f = fits.open(file)
    print(f)
    
    name = f[0].header['OBJNAME']
    ra = f[0].header['RA']
    dec = f[0].header['DEC']
    
    ms_wl = f[0].header['CRVAL1'] + f[0].header['CD1_1'] * np.arange(f[0].data.shape[2])
    ms_fl = f[0].data[0,0,:]
    ms_ivar = f[0].data[3,0,:]**-2
    
    wl, fl, ivar = continuum_normalize(ms_wl, ms_fl, ms_ivar)
    res, rv, e_rv, rvgrid, cc = msrvxc.fit.fit_rv(interp_bosz_norm, wvl, wl, fl, ivar, plot = False)

    names.append(name)
    ras.append(ra)
    decs.append(decs)
    rvs.append(rv)
    e_rvs.append(e_rv)

  0%|          | 0/11 [00:00<?, ?it/s]

NameError: name 'msrvxc' is not defined

In [ ]:
t = Table([names, ras, decs, rvs, e_rvs], names=('name', 'ms_ra', 'ms_dec', 'ms_rv', 'ms_e_rv'))

In [ ]:
names = list(set(t['name']))
ms_ras = []
ms_decs = []
rvs = []
e_rvs = []

for name in tqdm(names):
    ms_ras.append(t[t['name'] == name]['ms_ra'][0]
    ms_decs.append(t[t['name'] == name]['ms_dec'][0]
    
    rv = 0
    e_rv = 0
    count = 0
    
    for i in range(len(t[t['name'] == name])):
        if t[t['name'] == name]['ms_e_rv'][i] < 9999:
            count += 1
            rv += t[t['name'] == name]['ms_rv'][i]
            e_rv += t[t['name'] == name]['ms_e_rv'][i]**2
            
    rv = rv / count
    """
    
    IS THIS CORRECT? I DON'T KNOW!
    VVV
    
    """
    e_rv = np.sqrt(e_rv) / np.sqrt(count)
    
    rvs.append(rv)
    e_rvs.append(e_rv)

In [ ]:
ms_locs

In [ ]:
t2 = Table([names, ras, decs, rvs, e_rvs], names=('name', 'ms_ra', 'ms_dec', 'ms_rv', 'ms_e_rv'))

In [12]:
t2.write('ms_rvs.fits', overwrite = True)

In [35]:
t2

name,ms_loc,ms_rv,ms_e_rv
,"deg,deg",,
str11,SkyCoord,float64,float64
WD1323+1759,"200.82491666666664,17.994533333333333",-780.1161718274557,87.86655364813166
WD0833+2915,"128.47104166666665,29.25026111111111",-82.46178345224156,99.13201425372127
WD0650+3754,"102.59945833333332,37.91056666666667",-80.16460280724789,100.00499987500625
WD0325-0111,"51.294416666666656,-1.187438888888889",-41.86858469631204,100.00499987500625
WD1931+2109,"292.87941666666666,21.16358611111111",612.3459731416441,100.00499987500625
